In [0]:
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import geocoder
import time
print('Libraries imported.')

In [0]:
df = pd.read_csv('PO.csv')

In [0]:
df.head()

In [0]:
officename=[]
for i in df.index:
  district = df['regionname'][i]
  if district=="Mumbai":
    l1 = [df['officename'][i],df['pincode'][i],df['longitude'][i],df['latitude'][i]]
    officename.append(l1)

In [0]:
officename = pd.DataFrame(officename, columns=['officename','pincode'])

In [0]:
officename1 = officename.groupby(['pincode'])['officename'].apply(lambda x:','.join(x.astype(str))).reset_index()

In [0]:
from geopy.geocoders import Nominatim
loc = []
for i in pincode:
    geolocator = Nominatim(user_agent='my-application')
    time.sleep(.45)
    location = geolocator.geocode((str)(i)+"Mumbai India")
    print(i, location.latitude, location.longitude)
    loc.append([i,location.latitude, location.longitude])


In [0]:
temp = pd.DataFrame(loc)
temp.columns = [‘pincode’,’longitude’,’latitude’] 

In [0]:
officename1['longitude']=temp['longitude']
officename1['latitude']=temp['latitude']

In [0]:
address = 'Mumbai, Maharashtra'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [0]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)
count = 0
for lat, lng, neighborhood in zip(temp['latitude'], temp['longitude'], temp['pincode']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    li = [lat,lng]
    folium.CircleMarker(
        li,
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai) 
    count = count+1
map_mumbai

In [0]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20180605'

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [0]:
neighborhood_latitude = temp.loc[0, 'latitude'] 
neighborhood_longitude = temp.loc[0, 'longitude'] 

neighborhood_name = list(temp.loc[0, 'officename'].split(','))

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name[0], 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bazargate S.O are 18.9387711, 72.8353355.


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 15
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            

        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
mumbai_venues = getNearbyVenues(names=temp['officename'],
                                   latitudes=temp['latitude'],
                                   longitudes=temp['longitude']
                                  )

Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S.O,Stock Exchange S.O,Tajmahal S.O,Town Hall S.O (Mumbai),Mumbai G.P.O. 
Kalbadevi H.O,Ramwadi S.O,S. C. Court S.O,Thakurdwar S.O
B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Null Bazar S.O
Ambewadi S.O (Mumbai),Charni Road S.O,Chaupati S.O,Girgaon S.O,Madhavbaug S.O,Opera House S.O
Asvini S.O,Colaba Bazar S.O,Colaba S.O,Holiday Camp S.O,V.W.T.C. S.O
Malabar Hill S.O
Bharat Nagar S.O (Mumbai),Grant Road S.O,N.S.Patkar Marg S.O,S V Marg S.O,Tardeo S.O
Falkland Road S.O,J.J.Hospital S.O,Kamathipura S.O,M A Marg S.O,Mumbai Central H.O
Chinchbunder H.O,Noor Baug S.O,Princess Dock S.O
Dockyard Road S.O,Mazgaon Dock S.O,Mazgaon Road S.O,Mazgaon S.O,V K Bhavan S.O
Agripada S.O,BPC  Jacob Circle S.O,Chinchpokli S.O,Haines Road S.O,Jacob Circle S.O
BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin Institute S.O,Lal Baug S.O,Parel Naka S.O,Parel Rly Work Shop S.O,Parel S.O
Delisle Road S.O
Dadar Colony S.O,Dadar H.O,Naigaon S.O (Mumbai)
Sewri S.O
Kapad Ba

In [0]:
print(mumbai_venues.shape)
mumbai_venues.head()

(2275, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S....",18.938771,72.835335,Town House Cafe,18.938550,72.833464,Bar
1,"Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S....",18.938771,72.835335,Royal China,18.938715,72.832933,Chinese Restaurant
2,"Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S....",18.938771,72.835335,Cafe Excelsior,18.937701,72.833566,Café
3,"Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S....",18.938771,72.835335,Chhatrapati Shivaji Maharaj Terminus,18.940088,72.835257,Train Station
4,"Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S....",18.938771,72.835335,Sher-E-Punjab,18.937944,72.837853,Indian Restaurant


In [0]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A I Staff Colony S.O,Santacruz P&t Colony S.O",5,5,5,5,5,5
"Abitghar B.O,Abje B.O,Alonde B.O,Baliwali B.O,Dohe B.O,Gargoan B.O,Gorhe B.O,Hamarapur B.O,Kalambe B.O,Kanchad B.O,Khare Ambiwali B.O,Khariwali B.O,Kone B.O,Mala B.O,Malwada B.O,Maniwali B.O,Moj B.O,Parali B.O,Pik B.O,Posheri B.O,Sonale B.O,Tuse B.O,Utawali B.O,Varale B.O,Vilkos B.O,Wada S.O,Waki B.O",15,15,15,15,15,15
"Acchad B.O,Dabhadi B.O,Dhaniwari B.O,Dhundalwadi B.O,Dongri B.O,Girgaon B.O,Kochai B.O,Kurze B.O,Modgaon B.O,Talasari S.O,Udhave B.O,Uplat B.O,Vevji B.O,Zari B.O",15,15,15,15,15,15
"Achloli B.O,Khardi B.O,Kolose B.O,Konzar B.O,Mandle B.O,Nate S.O,Pachad B.O,Sandoshi B.O,Savrat B.O,Taloshi B.O,Tetghar B.O,Varandoli B.O,Wagheri B.O",4,4,4,4,4,4
"Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighode B.O,Gavan B.O,Jasai B.O,Jci Kamothe B.O,Jui B.O,Kelevane B.O,Khanda Colony S.O,Koproli B.O,Kundevahal B.O,Lodhivali B.O,Morbe B.O,Nere B.O,Nhava B.O,Panvel City S.O,Panvel H.O,Pargaon B.O,Somatane B.O,Ulwa B.O,Vahal B.O,Vaje B.O,Vasheni B.O,Wavandhal B.O,Wavarle B.O",13,13,13,13,13,13
...,...,...,...,...,...,...
Vidyashram S.O,15,15,15,15,15,15
Virar East S.O,15,15,15,15,15,15
Vishnunagar S.O,15,15,15,15,15,15


In [0]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 158 uniques categories.


In [0]:
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]
mumbai_onehot.head(15)

,Zoo,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,...,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Record Shop,Recreation Center,Rest Area,Restaurant,Roof Deck,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
mumbai_onehot.shape

(2275, 158)

In [0]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,Zoo,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,...,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Record Shop,Recreation Center,Rest Area,Restaurant,Roof Deck,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,"A I Staff Colony S.O,Santacruz P&t Colony S.O",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,"Abitghar B.O,Abje B.O,Alonde B.O,Baliwali B.O,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.066667,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,"Acchad B.O,Dabhadi B.O,Dhaniwari B.O,Dhundalwa...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.133333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,"Achloli B.O,Khardi B.O,Kolose B.O,Konzar B.O,M...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,"Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighod...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.076923,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.0,0.0,0.076923,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.076923,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Vidyashram S.O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.066667,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000

In [0]:
mumbai_grouped.shape

(209, 158)

In [0]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp1 = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp1.columns = ['venue','freq']
    temp1 = temp1.iloc[1:]
    temp1['freq'] = temp1['freq'].astype(float)
    temp1 = temp1.round({'freq': 2})
    print(temp1.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A I Staff Colony S.O,Santacruz P&t Colony S.O----
               venue  freq
0  Indian Restaurant   0.4
1        Coffee Shop   0.2
2      Historic Site   0.2
3  Electronics Store   0.2
4                Zoo   0.0


----Abitghar B.O,Abje B.O,Alonde B.O,Baliwali B.O,Dohe B.O,Gargoan B.O,Gorhe B.O,Hamarapur B.O,Kalambe B.O,Kanchad B.O,Khare Ambiwali B.O,Khariwali B.O,Kone B.O,Mala B.O,Malwada B.O,Maniwali B.O,Moj B.O,Parali B.O,Pik B.O,Posheri B.O,Sonale B.O,Tuse B.O,Utawali B.O,Varale B.O,Vilkos B.O,Wada S.O,Waki B.O----
                venue  freq
0                 Bar  0.13
1  Dim Sum Restaurant  0.07
2         Coffee Shop  0.07
3               Diner  0.07
4             Brewery  0.07


----Acchad B.O,Dabhadi B.O,Dhaniwari B.O,Dhundalwadi B.O,Dongri B.O,Girgaon B.O,Kochai B.O,Kurze B.O,Modgaon B.O,Talasari S.O,Udhave B.O,Uplat B.O,Vevji B.O,Zari B.O----
                  venue  freq
0           Pizza Place  0.13
1        Ice Cream Shop  0.13
2     Indian Restaurant  0.13
3  Fast Food

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"A I Staff Colony S.O,Santacruz P&t Colony S.O",Indian Restaurant,Historic Site,Electronics Store,Coffee Shop,Dim Sum Restaurant,Diner,Donut Shop,Dessert Shop,Department Store,English Restaurant
1,"Abitghar B.O,Abje B.O,Alonde B.O,Baliwali B.O,...",Bar,Brewery,Pizza Place,Lounge,Coffee Shop,Japanese Restaurant,Italian Restaurant,Irani Cafe,Diner,Dessert Shop
2,"Acchad B.O,Dabhadi B.O,Dhaniwari B.O,Dhundalwa...",Ice Cream Shop,Pizza Place,Fast Food Restaurant,Indian Restaurant,Gym,Lounge,Restaurant,Sports Bar,Coffee Shop,Department Store
3,"Achloli B.O,Khardi B.O,Kolose B.O,Konzar B.O,M...",Cupcake Shop,Chinese Restaurant,Garden,Bakery,Dim Sum Restaurant,Dessert Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
4,"Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighod...",Lounge,Indian Restaurant,Pub,Plaza,Food,Chinese Restaurant,Garden,Gas Station,Café,Tunnel


In [0]:
# set number of clusters
kclusters = 17

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)
print(mumbai_grouped_clustering)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)
kmeans.labels_[0:10] 

     Zoo  American Restaurant  ...  Women's Store  Yoga Studio
0    0.0                  0.0  ...            0.0          0.0
1    0.0                  0.0  ...            0.0          0.0
2    0.0                  0.0  ...            0.0          0.0
3    0.0                  0.0  ...            0.0          0.0
4    0.0                  0.0  ...            0.0          0.0
..   ...                  ...  ...            ...          ...
204  0.0                  0.0  ...            0.0          0.0
205  0.0                  0.0  ...            0.0          0.0
206  0.0                  0.0  ...            0.0          0.0
207  0.0                  0.0  ...            0.0          0.0
208  0.0                  0.0  ...            0.0          0.0

[209 rows x 157 columns]


array([6, 2, 3, 1, 0, 0, 2, 0, 0, 0], dtype=int32)

In [0]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
mumbai_merged = temp
mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='officename')
mumbai_merged.head() 

,pincode,officename,longitude,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,400001,"Bazargate S.O,Elephanta Caves Po B.O,M.P.T. S....",72.835335,18.938771,6.0,Indian Restaurant,Hotel,Café,Chinese Restaurant,Fast Food Restaurant,Irani Cafe,Bar,Athletics & Sports,Seafood Restaurant,Multiplex
1,400002,"Kalbadevi H.O,Ramwadi S.O,S. C. Court S.O,Thak...",72.826180,18.949540,6.0,Indian Restaurant,Train Station,Café,Fast Food Restaurant,Jewelry Store,Yoga Studio,Farm,Flea Market,Fish Market,Fish & Chips Shop
2,400003,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",72.834797,18.951606,12.0,Indian Restaurant,BBQ Joint,Restaurant,Rest Area,Middle Eastern Restaurant,Chinese Restaurant,Ice Cream Shop,Dessert Shop,Electronics Store,American Restaurant
3,400004,"Ambewadi S.O (Mumbai),Charni Road S.O,Chaupati...",72.821730,18.954329,3.0,Juice Bar,Fast Food Restaurant,Indian Restaurant,Ice Cream Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Train Station,Breakfast Spot,Athletics & Sports,Bakery
4,400005,"Asvini S.O,Colaba Bazar S.O,Colaba S.O,Holiday...",72.824315,18.916863,9.0,Indian Restaurant,Spa,Bar,Brewery,Men's Store,Middle Eastern Restaurant,Chinese Restaurant,Coffee Shop,Italian Restaurant,Restaurant


In [0]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['latitude'], mumbai_merged['longitude'], mumbai_merged['officename'], mumbai_merged['Cluster Labels']):
    if np.isnan(cluster):
        cluster = (int)(np.nan_to_num(cluster))
    else:
        cluster = (int)(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 6, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,pincode,officename,longitude,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,400055,"Santacruz(East) S.O,Vakola S.O",72.847201,19.081436,6.0,Bus Station,Outdoors & Recreation,Hotel,Asian Restaurant,Indian Restaurant,Farm,Food,Flea Market,Fish Market,Fish & Chips Shop
